## Transfer Learning Tuning:
  * Use Resnet50 to reduce model size and improve accuracy 
      * see [base model info comparison](https://medium.com/@sidereal/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5)
      * [resnet50 blog](https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33)
     
  * base_model: use RESNET50 and freeze at bottleneck layer (stop right before 2D AVERAGE POOL) 
  * top_model: tune dense layers (parameters are inspired by a few sources)
     * [source1](https://www.kaggle.com/suniliitb96/tutorial-keras-transfer-learning-with-resnet50)
     * [source2](https://www.kaggle.com/pmigdal/transfer-learning-with-resnet-50-in-keras)
     * [source3](https://towardsdatascience.com/transfer-learning-for-image-classification-using-keras-c47ccf09c8c8)



  
---
#### This cell is required in order to use GPU for running the model 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import keras
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras
keras.backend.get_session().run(tf.global_variables_initializer())  

Using TensorFlow backend.
W0904 23:45:39.479041 140378366863168 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0904 23:45:39.479885 140378366863168 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



---
#### Import necessary libraries

In [2]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model, model_from_json
from keras_preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#  import total dataframe

df = pd.read_pickle('../pickle_files/pic_df.pkl')

In [4]:
# subset a dataframe 
def create_test_train(df,label):
    '''
    df: dateframe of pic_ids 
    label: eyewear,hat, or beard, a string
    return: test and train df
    '''
    sub_set = df[['pic_id',label]]  # subset the label dataframe 
    X_train, X_test, y_train, y_test = train_test_split(sub_set['pic_id'], sub_set[label],
                                                        stratify = sub_set[label],
                                                        test_size = 0.2)
    df_train = pd.concat([X_train,y_train], axis=1)
    df_test = pd.concat([X_test,y_test], axis=1)
    return df_train, df_test
    

In [5]:
def create_generator(train_df, test_df,label,shuffle,batch_size,
                    rescale, preprocess_func, target_size,
                    class_mode, only_testing = False):
    '''
    train_df, test_df: dataframe for train and test 
    label: eyewear,hat, or beard, a string
    shuffle: weather to shuffle or not based upon feature extractor or not 
    batch_size: how many pictures per batch
    preprocess_func: vgg16 preprocess or ResNet50
    rescale: 1./255 or None
    target_size: (224,224) or (150,150)
    class_mode: none or binary 
    return:
    train generator and test generator 
    '''
    generators =[]
    if not only_testing:
        traingen = ImageDataGenerator(
            rescale = rescale,
            zoom_range= [0.8,1.7],
            shear_range=0.2,
            brightness_range=[0.5,1.5],
            rotation_range = 40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            preprocessing_function=preprocess_func)
        
        train_generator = traingen.flow_from_dataframe(
            dataframe=train_df,
            directory='../data/pics',
            x_col='pic_id',
            y_col=label,
            batch_size=batch_size,
            shuffle = shuffle,
            target_size=target_size,
            class_mode = class_mode)
        generators.append(train_generator)
        if class_mode:
        # create classweights for train 
            classweights = class_weight.compute_class_weight(
                'balanced',np.unique(train_generator.classes),train_generator.classes)
            generators.append(classweights)
    
    testgen = ImageDataGenerator(
        rescale = rescale,
        preprocessing_function=preprocess_func)
    
    test_generator = testgen.flow_from_dataframe(
        dataframe=test_df,
        directory='../data/pics',
        x_col='pic_id',
        y_col=label,
        batch_size=batch_size,
        shuffle=shuffle,
        target_size=target_size,
        class_mode=class_mode)
    generators.append(test_generator)
    
    return generators

In [6]:
train_df, test_df = create_test_train(df,'eyewear')

#### Get bottleneck features to tune top models

In [7]:
def save_bottleneck_features(df, label, batch_size, model_type, preprocess_func, rescale,
                            shuffle,target_size,class_mode):
    '''
    inputs:
    df: dataframe for all eyewear, hat, and beard 
    label: a string, eyewear, hat, or beard
    batch_size: process images in batches
    model_type: ResNet50 or VGG16
    shuffle: weather to shuffle or not based upon feature extractor or not 
    preprocess_func: vgg16 preprocess or ResNet50
    rescale: 1./255 or None
    target_size: (224,224) or (150,150)
    class_mode: none or binary 
    outputs:
    saves bottleneck features inside folder tuning_data as npy file
    '''
    # intialize the model, vgg16 or ResNet50 
    # make sure not to train the top layers 
    base_model = model_type(weights = 'imagenet',include_top = False)
    # generate test_train df
    train_df, test_df = create_test_train(df,label)
    # create train_generator and test_generator to get bottleneck inputs for train and test df 
    # make sure shuffle is False so we know the label follows the sequence of the dataframe 
    # so we can tune top_model 
    generators = create_generator(train_df=train_df,
                                                   test_df=test_df,
                                                   label=label,
                                                   shuffle=shuffle,
                                                   rescale=rescale,
                                                   preprocess_func=preprocess_input,
                                                   batch_size=16,
                                                   target_size=target_size,
                                                   class_mode=class_mode)  
    train_generator, test_generator = generators[0],generators[1]
    
    # get features saved as .npy in tunign_data folder 
    bottleneck_features_train = base_model.predict_generator(
        train_generator, train_df.shape[0]//batch_size)
    np.save(open(f'../tuning_data/resnet_data/untracked_resnet50/bottleneck_features_train_{label}_resnet50_beta.npy','wb'),
           bottleneck_features_train)
    
    bottleneck_features_test = base_model.predict_generator(
        test_generator, test_df.shape[0]//batch_size)
    np.save(open(f'../tuning_data/resnet_data/untracked_resnet50/bottleneck_features_test_{label}_resnet50_beta.npy','wb'),
           bottleneck_features_test)
    return train_df, test_df

#### Save bottleneck_features

In [8]:
train_df, test_df = save_bottleneck_features(df,'eyewear',16,ResNet50,preprocess_input,
                         None,False,(224,224),None)

W0904 23:46:11.958368 140378366863168 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0904 23:46:11.959231 140378366863168 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0904 23:46:11.962897 140378366863168 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4479: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0904 23:46:12.282516 140378366863168 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2041: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fus

Found 522 validated image filenames.
Found 131 validated image filenames.


In [9]:
def resnet50_model(input_shape, dropout):
    '''
    input_shape: input_shape for pooling layer 
    dropout:percentage for Dropout layer 
    optimizer: optimizer for model compiles 
    returns resnet50 model
    '''
    model = Sequential()
    model.add(GlobalAveragePooling2D(input_shape=input_shape))
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='sigmoid'))
    return model

In [ ]:
def vgg16_model(input_shape,dropout):
    '''
    input_shape: input_shape for pooling layer 
    dropout:percentage for Dropout layer 
    optimizer: optimizer for model compiles 
    returns vgg16 model
    '''
     # build top model
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))

    return model    

### Quick tuning of top models 

In [10]:
def train_top_model(train_df, test_df, epoch, batch_size, label, model_type,rescale,preprocess_func,
                   target_size):
    '''
    inputs:
    train_df, test_df: dataframes returned from save_bottleneck_features functions 
    epoch: num of epochs in fit 
    batch_size: same as image generator batch size 
    label: a string, eyewear, hat, or beard
    model_type: resnet50 or vgg16
    output:
    saves model weights in a folder 
    '''
    train_data = np.load(open(f'../tuning_data/resnet_data/untracked_resnet50/bottleneck_features_train_{label}_resnet50_beta.npy','rb'))
    # make sure train_data and train_label have same num of samples
    train_label = np.array(train_df[label].map({'0':0, '1':1}))[:-(train_df.shape[0]%batch_size)]
    
    test_data = np.load(open(f'../tuning_data/resnet_data/untracked_resnet50/bottleneck_features_test_{label}_resnet50_beta.npy','rb'))
    test_label = np.array(test_df[label].map({'0':0, '1':1}))[:-(test_df.shape[0]%batch_size)]
    
    # build top model
    if model_type == 'resnet50':
        model = resnet50_model(train_data.shape[1:],0.25)
    if model_type == 'vgg16':
        model = vgg16_model(train_data.shape[1:],0.5)
        
    model.compile(optimizer=optimizers.Adam(),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

    # checkpoint for best weights 
    filepath=f"../tuning_data/resnet_data/untracked_resnet50/best_bottleneck_resnet50_model_{label}_beta.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    
    classweight = create_generator(train_df, test_df,label,False,batch_size,
                    rescale, preprocess_func, target_size,
                    'binary', only_testing = False)[1]
    
    model.fit(train_data, train_label,
             epochs=epoch,
             batch_size=batch_size,
             validation_data=(test_data,test_label),
             callbacks=callbacks_list,
             class_weight = classweight)
    del model
    keras.backend.clear_session()

#### run train_top_model and save results in tuning_data

In [11]:
train_top_model(train_df, test_df, 50, 16, 'eyewear','resnet50',None,preprocess_input,target_size=(224,224))

W0904 23:46:37.476774 140378366863168 deprecation.py:506] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0904 23:46:37.494773 140378366863168 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0904 23:46:37.498836 140378366863168 deprecation.py:323] From /home/mindy/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as n

Found 522 validated image filenames belonging to 2 classes.
Found 131 validated image filenames belonging to 2 classes.
Train on 512 samples, validate on 128 samples
Epoch 1/50
512/512 [==============================] - 1s 2ms/step - loss: 0.5711 - acc: 0.7578 - val_loss: 0.4989 - val_acc: 0.7891

Epoch 00001: val_acc improved from -inf to 0.78906, saving model to ../tuning_data/resnet_data/untracked_resnet50/best_bottleneck_resnet50_model_eyewear_beta.h5
Epoch 2/50
512/512 [==============================] - 0s 295us/step - loss: 0.2263 - acc: 0.9023 - val_loss: 0.1716 - val_acc: 0.9062

Epoch 00002: val_acc improved from 0.78906 to 0.90625, saving model to ../tuning_data/resnet_data/untracked_resnet50/best_bottleneck_resnet50_model_eyewear_beta.h5
Epoch 3/50
512/512 [==============================] - 0s 295us/step - loss: 0.1132 - acc: 0.9648 - val_loss: 0.1632 - val_acc: 0.9531

Epoch 00003: val_acc improved from 0.90625 to 0.95312, saving model to ../tuning_data/resnet_data/untracke

512/512 [==============================] - 0s 296us/step - loss: 3.1186e-04 - acc: 1.0000 - val_loss: 0.2063 - val_acc: 0.9375

Epoch 00042: val_acc did not improve from 0.95312
Epoch 43/50
512/512 [==============================] - 0s 291us/step - loss: 3.2025e-04 - acc: 1.0000 - val_loss: 0.2054 - val_acc: 0.9453

Epoch 00043: val_acc did not improve from 0.95312
Epoch 44/50
512/512 [==============================] - 0s 293us/step - loss: 2.5207e-04 - acc: 1.0000 - val_loss: 0.2055 - val_acc: 0.9375

Epoch 00044: val_acc did not improve from 0.95312
Epoch 45/50
512/512 [==============================] - 0s 303us/step - loss: 2.2296e-04 - acc: 1.0000 - val_loss: 0.2075 - val_acc: 0.9375

Epoch 00045: val_acc did not improve from 0.95312
Epoch 46/50
512/512 [==============================] - 0s 292us/step - loss: 2.8903e-04 - acc: 1.0000 - val_loss: 0.2093 - val_acc: 0.9453

Epoch 00046: val_acc did not improve from 0.95312
Epoch 47/50
512/512 [==============================] - 0s 299u

### Fine Tune Top Model to improve accuracy

In [12]:
def fine_tune_model(train_df, test_df,epoch, batch_size,label, 
                    model_type,dim, rescale, preprocess_func,print_model = False):
    '''
    inputs:
    train_df, test_df: dataframes returned from save_bottleneck_features functions 
    epoch: num of epochs in fit 
    batch_size: same as image generator batch size 
    label: a string, eyewear, hat, or beard
    model_type: resnet50 or vgg16
    dim: 224 or 150 
    rescale: None resnet50, 1./255 for vgg16
    preprocess_func: depends on resnet or vgg16
    return model 
    '''
    # build model and freeze top layers
    # input_shape: width, height, RGB (from image generator)
    if model_type == 'resnet50':
        base_model = ResNet50(weights='imagenet',include_top=False, input_shape=(dim,dim,3))
        # build top model
        top_model = resnet50_model(base_model.output_shape[1:],0.25)
    if model_type == 'vgg16':
        base_model = VGG16(weights='imagenet',include_top=False, input_shape=(dim,dim,3))
        top_model = vgg16_model(base_model.output_shape[1:],0.5)
    # load saved weights to fine tune parameters 
    top_model.load_weights(f'../tuning_data/resnet_data/untracked_resnet50/best_bottleneck_resnet50_model_{label}_beta.h5')
    # add top model to model
    model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
    # we will tune last 8 layers of the model: activation49 and fully connected layer 
    for layer in model.layers[:-5]:
        layer.trainable = False
    # we can tune the parameters for lr and momentum later to get better results
    model.compile(loss='binary_crossentropy',
             optimizer=optimizers.SGD(lr=5e-5, momentum = 0.9),
             metrics=['accuracy'])
    # prepare train generator using data augmentation to battle small sample size 
    generators = create_generator(train_df,test_df, label,True,16,rescale,
                                                    preprocess_func,(dim,dim),
                                                    'binary')
    train_generator, classweight, test_generator = generators[0], generators[1], generators[2]
    
    # checkpoint for best weights 
    filepath=f"../tuning_data/resnet_data/untracked_resnet50/best_resnet50_model_5_layer_{label}_0.25dobeta_2.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    
    # run and fit model 
    result = model.fit_generator(
    train_generator,
    steps_per_epoch=train_df.shape[0]//batch_size,
    epochs=epoch,
    validation_data=test_generator,
    validation_steps=test_df.shape[0]//batch_size,
    verbose=1,class_weight=list(classweight),
    callbacks=callbacks_list
    )
    
    if print_model:
        model.summary()
    
    del model
    keras.backend.clear_session()
    return result                

### save model history 

In [ ]:
model_history = fine_tune_model(train_df,test_df,50,16,'eyewear','resnet50',
                                224,None,preprocess_input)

Found 522 validated image filenames belonging to 2 classes.
Found 131 validated image filenames belonging to 2 classes.
Epoch 1/50
32/32 [==============================] - 8s 240ms/step - loss: 0.3431 - acc: 0.8484 - val_loss: 0.1550 - val_acc: 0.9375

Epoch 00001: val_acc improved from -inf to 0.93750, saving model to ../tuning_data/resnet_data/untracked_resnet50/best_resnet50_model_5_layer_eyewear_0.25dobeta_2.h5
Epoch 2/50
32/32 [==============================] - 8s 241ms/step - loss: 0.3027 - acc: 0.8847 - val_loss: 0.1928 - val_acc: 0.9217

Epoch 00002: val_acc did not improve from 0.93750
Epoch 3/50
32/32 [==============================] - 8s 248ms/step - loss: 0.2489 - acc: 0.9106 - val_loss: 0.1807 - val_acc: 0.9217

Epoch 00003: val_acc did not improve from 0.93750
Epoch 4/50
32/32 [==============================] - 7s 223ms/step - loss: 0.2516 - acc: 0.8981 - val_loss: 0.1292 - val_acc: 0.9391

Epoch 00004: val_acc improved from 0.93750 to 0.93913, saving model to ../tuning_d

In [ ]:
highest_val_acc, highest_train_acc = max(model_history.history['val_acc']), max(model_history.history['acc'])
print(f'highest test accuracy: {highest_val_acc}')
print('------------------')
print(f'highest train accuracy: {highest_train_acc}')

In [ ]:
lowest_val_loss, lowest_train_loss = min(model_history.history['val_loss']), min(model_history.history['loss'])
print(f'lowest test loss: {lowest_val_loss}')
print('------------------')
print(f'lowest train loss: {lowest_train_loss}')

In [ ]:
plt.plot(model_history.history['val_acc'], color = 'red', label = 'test')
plt.plot(model_history.history['acc'], color = 'blue', label = 'train')
plt.title('accuracy')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(model_history.history['val_loss'], color = 'red', label = 'test')
plt.plot(model_history.history['loss'], color = 'blue', label = 'train')
plt.title('loss')
plt.ylabel('loss')
plt.legend()
plt.show()

### Save model 

In [ ]:
# # save model artchitecture 
# resnet50_model_8_up = model_history.model
# resnet50_model_8_up_json = resnet50_model_8_up.to_json()

In [ ]:
# # serialize model to JSON
# with open("../tuning_data/resnet_data/resnet50_model_8_up.json", "w") as json_file:
#     json_file.write(resnet50_model_8_up_json)

### Testing model

In [ ]:
# load json and create model
def model_testing(df, model_path, weight_path, label, shuffle = False,
                 batch_size = 32, rescale=None,preprocess_func=preprocess_input,
                 target_size = (224,224),class_mode ='binary', only_testing = True):
    '''
    df: the entire picture df 
    model_path: json model path
    weight_path: path for weights 
    label: eyewear, hat, or beard
    '''
    # check the whole dataframe
    sub_set = df[['pic_id',label]] 
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load model weight yewear_decay_
    loaded_model.load_weights(weight_path)
    # test test data
    data_generator= create_generator(None, df,label,shuffle,batch_size,
                    rescale, preprocess_func, target_size,
                    class_mode, only_testing = only_testing)
    data_generator=data_generator[0]
    labels = df[label].map({'1':1, '0':0})
    
    loaded_model.compile(loss='binary_crossentropy',
             optimizer=optimizers.SGD(),
             metrics=['accuracy']
                        )
    y_pred = np.around(loaded_model.predict_generator(data_generator, workers=8))
    y_pred = [1-x for x in y_pred] # for old model 
    fpr, tpr, thresholds = roc_curve(labels, y_pred)
    area = roc_auc_score(labels, y_pred)
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % area)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    print(confusion_matrix(labels, y_pred))
    print(classification_report(labels, y_pred))    

### Eyewear

In [ ]:
model_testing(df=df,model_path='../tuning_data/resnet_data/resnet50_model_5_up.json',
             weight_path='../tuning_data/resnet_data/untracked_resnet50/best_resnet50_model_5_layer_eyewear_0.25dobeta.h5',
              label='eyewear')
# use best_resnet50_model_5_layer_eyewear_0.25dobeta_2.h5

In [ ]:
model_testing(df=df,model_path='../tuning_data/resnet_data/resnet50_model_5_up.json',
             weight_path='../tuning_data/resnet_data/untracked_resnet50/best_resnet50_model_5_layer_eyewear_0.25dobeta_2.h5',
              label='eyewear')
# best model 

In [ ]:
model_testing(df,'../tuning_data/resnet_data/resnet50_model_5_up.json', 
              '../tuning_data/resnet_data/untracked_resnet50/best_resnet50_model_eyewear.h5',
              'eyewear')

### Testing hat 

In [ ]:
model_testing(df=df,model_path='../tuning_data/resnet_data/resnet50_model_5_up.json',
             weight_path='../tuning_data/resnet_data/untracked_resnet50/best_resnet50_model_5_layer_hat_0.25dobeta_2.h5',
              label='hat')

In [ ]:
model_testing(df,'../tuning_data/resnet_data/resnet50_model_5_up.json', 
              '../tuning_data/resnet_data/untracked_resnet50/best_resnet50_model_hat.h5',
             'hat')

# best resnet

### Testing beard

In [ ]:
model_testing(df=df,model_path='../tuning_data/resnet_data/resnet50_model_5_up.json',
             weight_path='../tuning_data/resnet_data/untracked_resnet50/best_resnet50_model_5_layer_beard_0.25dobeta_2.h5',
              label='beard')

In [ ]:
model_testing(df,'../tuning_data/resnet_data/resnet50_model_5_up.json',
              '../tuning_data/resnet_data/untracked_resnet50/best_resnet50_model_beard.h5','beard')